> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [ ]:
import pandas as pddf = pd.read_csv('salaries.csv')df.head()

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [ ]:
from sqlalchemy import Column, Integer, String, Float, Date, Enumfrom sqlalchemy.ext.declarative import declarative_baseimport enumclass SexoEnum(enum.Enum):    F = 'F'    M = 'M'class CargoEnum(enum.Enum):    ANALYST = 'Analyst'    ASSOCIATE = 'Associate'    SENIOR_ANALYST = 'Senior Analyst'    SENIOR_MANAGER = 'Senior Manager'    MANAGER = 'Manager'    DIRECTOR = 'Director'class UnidadeEnum(enum.Enum):    FINANCE = 'Finance'    WEB = 'Web'    IT = 'IT'    OPERATIONS = 'Operations'    MARKETING = 'Marketing'    MANAGEMENT = 'Management'Base = declarative_base()class Empregado(Base):    __tablename__ = 'salarios'        id = Column(Integer, primary_key=True, autoincrement=True)    primeiro_nome = Column(String)    sobrenome = Column(String)    sexo = Column(Enum(SexoEnum))    data_contratacao = Column(Date)    data_atual = Column(Date)    cargo = Column(Enum(CargoEnum))    idade = Column(Integer)    salario = Column(Float)    unidade = Column(Enum(UnidadeEnum))    ferias_usadas = Column(Integer)    ferias_restantes = Column(Integer)    avaliacoes = Column(Float)    experiencia_anterior = Column(Float)

#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [ ]:
from sqlalchemy import create_engineengine = create_engine('sqlite:///salarios.db')

#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [ ]:
Base.metadata.create_all(engine)

#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [ ]:
df.to_sql('salarios', engine, if_exists='append', index=False)

#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [ ]:
from sqlalchemy import textwith engine.connect() as conn:    resultado = conn.execute(text("SELECT CARGO, MIN(SALARIO/12) as salario_min, MAX(SALARIO/12) as salario_max, AVG(SALARIO/12) as salario_med FROM salarios GROUP BY CARGO"))    print(resultado.fetchall())

In [ ]:
sql = "SELECT CARGO, MIN(SALARIO/12) as salario_min, MAX(SALARIO/12) as salario_max, AVG(SALARIO/12) as salario_med FROM salarios GROUP BY CARGO"df_resultado = pd.read_sql_query(sql, engine.connect())print(df_resultado)

In [ ]:
from sqlalchemy.orm import Sessionfrom sqlalchemy import select, funcwith Session(engine) as session:    stmt = select(        Empregado.cargo,        func.min(Empregado.salario / 12).label('salario_min'),        func.max(Empregado.salario / 12).label('salario_max'),        func.avg(Empregado.salario / 12).label('salario_med')    ).group_by(Empregado.cargo)    resultados = session.execute(stmt).all()    for linha in resultados:        print(linha)